# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key


### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [33]:
cities_df = pd.read_csv("../output_data/my_cities.csv", index_col='City ID')
cities_df.head()

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
City ID,,,,,,,,,
0,Henties Bay,0,NaN,1585689020,90,-22.12,14.28,15.00,0.89
1,Bethel,90,US,1585688890,61,41.37,-73.41,9.44,1.50
2,Saint-François,0,FR,1585689096,51,46.42,3.91,3.00,2.10
3,Ushuaia,40,AR,1585689028,66,-54.80,-68.30,10.00,5.10
4,Mar del Plata,40,AR,1585688808,55,-38.00,-57.56,18.89,7.20


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [4]:
gmaps.configure(api_key=g_key)

In [63]:
locations = cities_df[['Lat', 'Lng']]
fig = gmaps.figure()
heat_layer = gmaps.heatmap_layer(locations, weights=cities_df['Humidity'], 
                                 dissipating=False, max_intensity=cities_df['Humidity'].max(),
                                 point_radius = 2)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [78]:
# My values are in metric. Need to convert temperature and wind speed.
temp_max_metric = (80-32) * 5/9 
temp_min_metric = (70-32) * 5/9 
wind_metric = 10 / 2.237
cities_ideal_df = cities_df.dropna()
cities_ideal_df = cities_ideal_df.loc[
    (cities_ideal_df['Max Temp'] < temp_max_metric) &
    (cities_ideal_df['Max Temp'] > temp_min_metric) &
    (cities_ideal_df['Cloudiness'] == 0) &
    (cities_ideal_df['Wind Speed'] < wind_metric), :
]
print(f'{len(cities_ideal_df)} cities meet your criteria.')
cities_ideal_df.head()

9 cities meet your criteria.


,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
City ID,,,,,,,,,
55,Inhambane,0,MZ,1585689112,70,-23.86,35.38,23.41,4.07
141,Makkah al Mukarramah,0,SA,1585688854,39,21.43,39.83,25.11,0.81
175,Cockburn Town,0,TC,1585688992,68,21.46,-71.14,25.08,4.36
198,Abu Dhabi,0,AE,1585689432,60,24.47,54.37,23.89,1.50
203,Sharjah city,0,AE,1585688982,73,25.36,55.40,23.89,3.10


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [79]:
hotel_df = cities_ideal_df.reset_index(drop=True)
hotel_df['Hotel Name'] = ""
hotel_df

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed,Hotel Name
0,Inhambane,0,MZ,1585689112,70,-23.86,35.38,23.41,4.07,
1,Makkah al Mukarramah,0,SA,1585688854,39,21.43,39.83,25.11,0.81,
2,Cockburn Town,0,TC,1585688992,68,21.46,-71.14,25.08,4.36,
3,Abu Dhabi,0,AE,1585689432,60,24.47,54.37,23.89,1.50,
4,Sharjah city,0,AE,1585688982,73,25.36,55.40,23.89,3.10,
5,Khipro,0,PK,1585689204,56,25.83,69.38,25.02,4.01,
6,Tessalit,0,ML,1585689221,11,20.20,1.01,25.88,4.30,
7,Kyaukse,0,MM,1585689229,36,21.60,96.13,26.21,3.83,
8,Dhāri,0,IN,1585689251,51,21.33,71.02,21.79,2.96,


In [80]:
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
params = {
    "rankby": "distance",
    "keyword": "hotel",
    "key": g_key,
}
# use iterrows to iterate through pandas dataframe
for index, row in hotel_df.iterrows():

    # add location to params dict
    params['location'] = f"{row['Lat']}, {row['Lng']}"

    # assemble url and make API request
    print(f"Retrieving Results for Index {index}: {row['City']}.")
    response = requests.get(base_url, params=params).json()
    
    # extract results
    results = response['results']
    
    try:
        print(f"Closest hotel is {results[0]['name']}.")
        hotel_df.loc[index, 'Hotel Name'] = results[0]['name']
        
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
        
    print("------------")

Retrieving Results for Index 0: Inhambane.
Closest hotel is Hotel Casa Do Capitão.
------------
Retrieving Results for Index 1: Makkah al Mukarramah.
Closest hotel is OYO 375 Deyar Alrawada Hotel.
------------
Retrieving Results for Index 2: Cockburn Town.
Closest hotel is Screaming Reels Guest House.
------------
Retrieving Results for Index 3: Abu Dhabi.
Closest hotel is Marriott Executive Apartments Downtown Abu Dhabi.
------------
Retrieving Results for Index 4: Sharjah city.
Closest hotel is DANA HOTEL.
------------
Retrieving Results for Index 5: Khipro.
Closest hotel is Raja Hotel.
------------
Retrieving Results for Index 6: Tessalit.
Missing field/result... skipping.
------------
Retrieving Results for Index 7: Kyaukse.
Closest hotel is Hotel Kyaukse.
------------
Retrieving Results for Index 8: Dhāri.
Closest hotel is The Leonia Heritage.
------------


In [82]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name

hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [83]:
# Add marker layer ontop of heat map
# Assign the marker layer to a variable
markers = gmaps.symbol_layer(locations, info_box_content=hotel_info)
# Add the layer to the map
fig.add_layer(markers)
fig.add_layer(heat_layer)
fig

# Display Map

Figure(layout=FigureLayout(height='420px'))